In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import time

In [2]:
df1 = pd.read_csv('cboesymboldirequityindex.csv')
df2 = pd.read_csv('cboesymboldirweeklys.csv')

In [3]:
df1

,Company Name,Stock Symbol,DPM Name,Post/Station
0,-1x Short VIX Futures ETF,SVIX,Citadel Securities LLC,9/1
1,1 800 FLOWERS COM INC CL A,FLWS,Belvedere Trading LLC,6/1
2,10X CAP VENTURE ACQSTN CORP II COM CL A,VCXA,Belvedere Trading LLC,4/1
3,10X GENOMICS INC CL A COM,TXG,"Susquehanna Securities, LLC",4/1
4,17 ED & TECHNOLOGY GROUP INC ADS,YQ,Belvedere Trading LLC,4/1
...,...,...,...,...
5037,ZUORA INC COM CL A,ZUO,"Susquehanna Securities, LLC",3/1
5038,ZURN ELKAY WATER SOLNS CORP COM,ZWS,"Susquehanna Securities, LLC",3/1
5039,ZYMEWORKS INC COM,ZYME,Citadel Securities LLC,2/1
5040,ZYNERBA PHARMACEUTICALS INC COM,ZYNE,"Susquehanna Securities, LLC",5/1


In [4]:
df2

,Company Name,Stock Symbol
0,2x Long VIX Futures ETF,UVIX
1,3M CO COM,MMM
2,ABBOTT LABS COM,ABT
3,ABBVIE INC COM,ABBV
4,ABERCROMBIE & FITCH CO CL A,ANF
...,...,...
622,ZOOM VIDEO COMMUNICATIONS INC CL A,ZM
623,ZSCALER INC COM,ZS
624,iPath Series B S&P 500 VIX Short-Term Futures ETN,VXX
625,iShares MSCI India ETF,INDA


In [5]:
df1.columns

Index(['Company Name', ' Stock Symbol', ' DPM Name', ' Post/Station'], dtype='object')

In [6]:
# Get all the 'Stock Symbol' from df1 and df2
# and put them into a list
lst1 = df1[' Stock Symbol'].tolist()
lst2 = df2[' Stock Symbol'].tolist()
lst = lst1 + lst2

In [7]:
lst

['SVIX',
 'FLWS',
 'VCXA',
 'TXG',
 'YQ',
 'TURN',
 'ATNF',
 'DIBS',
 'XXII',
 'ME',
 'TSVT',
 'TWOU',
 'BITX',
 'UVIX',
 'DDD',
 'MMM',
 'FDMT',
 'FEAM',
 'ETNB',
 'EGHT',
 'NMTRQ',
 'MASS',
 'AKA',
 'AMRK',
 'ATEN',
 'AAON',
 'AIR',
 'ABT',
 'ABBV',
 'ABCM',
 'ABCL',
 'ABEO',
 'ANF',
 'ABM',
 'BCIM',
 'BCI',
 'BCD',
 'SGOL',
 'SIVR',
 'ABSI',
 'ACTG',
 'ASO',
 'ACHC',
 'ACAD',
 'ACEL',
 'AXDX',
 'ACN',
 'ACCO',
 'ACCD',
 'ARAY',
 'ACRX',
 'SLRN',
 'ACER',
 'ACHV',
 'ACIW',
 'ACRS',
 'ACMR',
 'ACNB',
 'ACOR',
 'ACR',
 'ATNM',
 'ATVI',
 'AYI',
 'ABOS',
 'GOLF',
 'AFIB',
 'ACVA',
 'PEO',
 'ADMP',
 'AHCO',
 'ADAP',
 'ADPT',
 'ADCT',
 'ADUS',
 'AGRO',
 'ADEA',
 'ADIL',
 'ACET',
 'ADNT',
 'ADTX',
 'ADMA',
 'ADBE',
 'ADT',
 'ATGE',
 'ADTN',
 'AAP',
 'WMS',
 'AEIS',
 'AMD',
 'ASIX',
 'ADV',
 'ADN',
 'ADVM',
 'EATV',
 'GK',
 'LETB',
 'MSOX',
 'YOLO',
 'MSOS',
 'HDGE',
 'ACM',
 'AGLE',
 'AEG',
 'AEHR',
 'AMTX',
 'AER',
 'AVAV',
 'ASLE',
 'AES',
 'AEMD',
 'AEVA',
 'LIDR',
 'AFCG',
 'AMG',
 'AFM

In [8]:
# Remove any Tickers that contain '.' in lst
lst = [x for x in lst if '.' not in x]

In [9]:
from matplotlib.pylab import norm


def black_scholes_greeks(S, X, T, r, sigma, option_type='put'):
    d1 = (np.log(S / X) + (r + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)
    
    N_d1_prime = norm.pdf(d1)
    
    if option_type == 'call':
        C = S * N_d1 - X * np.exp(-r * T) * N_d2
        Delta = N_d1
        Gamma = N_d1_prime / (S * sigma * np.sqrt(T))
        Theta = -(S * N_d1_prime * sigma / (2 * np.sqrt(T))) - r * X * np.exp(-r * T) * N_d2
        Vega = S * np.sqrt(T) * N_d1_prime
        Rho = X * T * np.exp(-r * T) * N_d2
    elif option_type == 'put':
        C = X * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        Delta = N_d1 - 1
        Gamma = N_d1_prime / (S * sigma * np.sqrt(T))
        Theta = -(S * N_d1_prime * sigma / (2 * np.sqrt(T))) + r * X * np.exp(-r * T) * norm.cdf(-d2)
        Vega = S * np.sqrt(T) * N_d1_prime
        Rho = -X * T * np.exp(-r * T) * norm.cdf(-d2)
    else:
        raise ValueError("Invalid option type. Use either 'call' or 'put'.")
        
    return [C, Delta, Gamma, Theta, Vega, Rho]

In [10]:
import datetime


def fetch_data(ticker, expiration_date):
    try:
        opts = yf.Ticker(ticker).option_chain(expiration_date)
        puts = opts.puts

        if 'bid' not in puts.columns:
            print(f'No options data for {ticker} on {expiration_date}')
            return pd.DataFrame()

        stock_price = round(yf.Ticker(ticker).info['currentPrice'], 2)
        condition = (puts['strike'] < stock_price) & (puts['bid'] > 0)
        puts = puts[condition].copy()
        stock_info = yf.Ticker(ticker).info

        # Add the ETF ticker to the DataFrame
        puts['ETF'] = ticker

        # Add the Stock Price to the DataFrame
        puts['Stock Price'] = stock_price
        # Add how out of the money the put is
        puts['OTM (%)'] = (((puts['Stock Price'] - puts['strike']) / puts['Stock Price']) * 100).round(2)
        # Add the expiration date to the DataFrame
        puts['Expiration Date'] = expiration_date
        # Add 'ROI (%)' column to DataFrame
        puts['ROI (%)'] = ((puts['bid'] / (puts['strike'] - puts['bid'])) * 100).round(2)
        # Add 'highPrice' column to DataFrame
        puts['highPrice'] = puts['bid']
        # Add 'Original ROI (%)' column to DataFrame
        puts['Original ROI (%)'] = ((puts['bid'] / (puts['strike'] - puts['bid'])) * 100).round(2)
        # Add 'Original OTM (%)' column to DataFrame
        puts['Original OTM (%)'] = (((puts['Stock Price'] - puts['strike']) / puts['Stock Price']) * 100).round(2)
        # Add 'Implied Volatility' column to DataFrame
        puts['Implied Volatility'] = puts['impliedVolatility'].round(2)
        # Add 'Original Implied Volatility' column to DataFrame
        puts['Original Implied Volatility'] = puts['impliedVolatility'].round(2)
        # Add 'Original Stock Price' column to DataFrame
        puts['Original Stock Price'] = stock_price
        # Add 'targetHighPrice' column to DataFrame
        puts['targetHighPrice'] = stock_info['targetHighPrice']
        # Add targetLowPrice' column to DataFrame
        puts['targetLowPrice'] = stock_info['targetLowPrice']
        # Add 'targetMeanPrice' column to DataFrame
        puts['targetMeanPrice'] = stock_info['targetMeanPrice']
        # Add 'targetMedianPrice' column to DataFrame
        puts['targetMedianPrice'] = stock_info['targetMedianPrice']
        # Add 'recommendationMean' column to DataFrame
        puts['recommendationMean'] = stock_info['recommendationMean']
        # Add 'recommendationKey' column to DataFrame
        puts['recommendationKey'] = stock_info['recommendationKey']

        # Calculate the number of days until expiration
        today = datetime.now()
        puts['daysToExpiration'] = (pd.to_datetime(puts['Expiration Date']) - today).dt.days

        # Calculate the Greeks
        greeks = black_scholes_greeks(puts['Stock Price'], puts['strike'], puts['daysToExpiration'] / 365, 0.01, puts['impliedVolatility'], option_type='put')
        # Add 'Delta' column to DataFrame
        puts['Delta'] = greeks[1].round(2)
        # Add 'Gamma' column to DataFrame
        puts['Gamma'] = greeks[2].round(2)
        # Add 'Theta' column to DataFrame
        puts['Theta'] = greeks[3].round(2)
        # Add 'Vega' column to DataFrame
        puts['Vega'] = greeks[4].round(2)
        # Add 'Rho' column to DataFrame
        puts['Rho'] = greeks[5].round(2)

        # Fetch historical data for the ETF ticker
        historical_data = yf.Ticker(ticker).history(period="1y")

        # Calculate current moving averages
        current_50_day_MA = historical_data['Close'].rolling(window=50).mean().iloc[-1]
        current_100_day_MA = historical_data['Close'].rolling(window=100).mean().iloc[-1]
        current_200_day_MA = historical_data['Close'].rolling(window=200).mean().iloc[-1]

        # Calculate current RSI
        delta = historical_data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean().iloc[-1]
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean().iloc[-1]
        current_RSI = 100 - (100 / (1 + (gain / loss)))

        # Calculate current MACD
        short_ema = historical_data['Close'].ewm(span=12, adjust=False).mean().iloc[-1]
        long_ema = historical_data['Close'].ewm(span=26, adjust=False).mean().iloc[-1]
        current_MACD = short_ema - long_ema

        # Calculate current VWAP
        typical_price = (historical_data['High'] + historical_data['Low'] + historical_data['Close']) / 3
        current_VWAP = np.cumsum(typical_price * historical_data['Volume']).iloc[-1] / np.cumsum(historical_data['Volume']).iloc[-1]

        # Add single value indicators to puts DataFrame
        puts['50-day MA'] = current_50_day_MA.round(2)
        puts['100-day MA'] = current_100_day_MA.round(2)
        puts['200-day MA'] = current_200_day_MA.round(2)
        puts['RSI'] = current_RSI.round(2)
        puts['MACD'] = current_MACD.round(2)
        puts['VWAP'] = current_VWAP.round(2)
        
        # Calculate 52-Week High and 52-Week Low
        high_52_week = historical_data['Close'].max()
        low_52_week = historical_data['Close'].min()

        puts['52WeekHigh'] = high_52_week.round(2)
        puts['52WeekLow'] = low_52_week.round(2)

        return puts
    except Exception as e:
        print(f'Error fetching data for {ticker} on {expiration_date}: {e}')

In [16]:
print(lst)

['SVIX', 'FLWS', 'VCXA', 'TXG', 'YQ', 'TURN', 'ATNF', 'DIBS', 'XXII', 'ME', 'TSVT', 'TWOU', 'BITX', 'UVIX', 'DDD', 'MMM', 'FDMT', 'FEAM', 'ETNB', 'EGHT', 'NMTRQ', 'MASS', 'AKA', 'AMRK', 'ATEN', 'AAON', 'AIR', 'ABT', 'ABBV', 'ABCM', 'ABCL', 'ABEO', 'ANF', 'ABM', 'BCIM', 'BCI', 'BCD', 'SGOL', 'SIVR', 'ABSI', 'ACTG', 'ASO', 'ACHC', 'ACAD', 'ACEL', 'AXDX', 'ACN', 'ACCO', 'ACCD', 'ARAY', 'ACRX', 'SLRN', 'ACER', 'ACHV', 'ACIW', 'ACRS', 'ACMR', 'ACNB', 'ACOR', 'ACR', 'ATNM', 'ATVI', 'AYI', 'ABOS', 'GOLF', 'AFIB', 'ACVA', 'PEO', 'ADMP', 'AHCO', 'ADAP', 'ADPT', 'ADCT', 'ADUS', 'AGRO', 'ADEA', 'ADIL', 'ACET', 'ADNT', 'ADTX', 'ADMA', 'ADBE', 'ADT', 'ATGE', 'ADTN', 'AAP', 'WMS', 'AEIS', 'AMD', 'ASIX', 'ADV', 'ADN', 'ADVM', 'EATV', 'GK', 'LETB', 'MSOX', 'YOLO', 'MSOS', 'HDGE', 'ACM', 'AGLE', 'AEG', 'AEHR', 'AMTX', 'AER', 'AVAV', 'ASLE', 'AES', 'AEMD', 'AEVA', 'LIDR', 'AFCG', 'AMG', 'AFMD', 'AFRM', 'AFL', 'MITT', 'AGCO', 'UAVS', 'AGEN', 'A', 'AGTI', 'AGL', 'AGYS', 'AGIO', 'AGNC', 'AEM', 'API', 'ADC'

In [ ]:
# print lst neatly